In [1]:
# comments and details to come !
import time
import numpy as np
import scipy as sp
import imageio
import os
from IPython.display import display, clear_output, HTML
import sys
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'

from mcClass import *
import matplotlib.pyplot as plt
from matplotlib import animation, rc
animation.rcParams['animation.writer']='ffmpeg'

%matplotlib inline

In [2]:
fM = 1.0
fS = 0.5
octa = 1
th = 20
thS = 20
fT = 1.0/0.1
v = 2.0*np.array([np.cos(th),np.sin(th)])

MC = motionCloud(overSamp=12, N=512, chooseDev=0, show=1, stdConst=35)
MC.mcKernel(fM, fS, th, thS, fT, v, octa)
MC.initGPU()
            

Correct parameters LifeTime = 0.100000 > 0.075406 



/usr/lib/python3/dist-packages/pyopencl/__init__.py:63: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [4]:
# First set up the figure, the axis, and the plot element we want to animate
fig01, axes01 = plt.subplots(nrows=1, ncols=1,figsize=(5, 5))
plt.close()

def init():
    return ()


def animate(i): 
    
    im0 = axes01.imshow(MC.getFrame(), cmap='gray')
    axes01.xaxis.set_visible(False)
    axes01.yaxis.set_visible(False)
    
    clear_output(wait=True)
    print('It: %i'%i)
    sys.stdout.flush()
    return (im0,)
            
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig01, animate, init_func=init, frames=100, interval=50, blit=True)
HTML(anim.to_html5_video()) 


It: 99


In [ ]:
im = np.zeros((512,512,3,128), dtype=np.uint8)

for i in range(128):
    frame = MC.getFrame() + 127.5
    frame[frame>255]=255
    frame[frame<0]=0
    im[:,:,0,i] = np.uint8(frame)
    im[:,:,1,i] = np.uint8(frame)
    im[:,:,2,i] = np.uint8(frame)
    
wvid = imageio.get_writer('mc.avi', fps=20.0)

for i in range(128):
    wvid.append_data(im[:,:,:,i])
    
wvid.close()

In [87]:
del MC
N = 256
Nf = 400
trajSamples = np.linspace(0, 2*np.pi, Nf)

fM = 1.0 + 2.0/(2*np.pi)*trajSamples
fS = 0.25
octa = 1
th = 0.0 + 100.0/(2*np.pi)*trajSamples
thS = 2.0 + 50.0/(2*np.pi)*trajSamples
fT = 1.0/0.05
th0 = th[0]
v = 0.0*np.array([np.cos(th0),np.sin(th0)])

MC = motionCloud(overSamp=40, N=N, chooseDev=0, show=0, stdConst=35)
MC.mcKernel(fM[0], fS, th[0], thS[0], fT, v, octa)
MC.initGPU()
            
for i in range(200):
    MC.getFrame()

In [88]:
im = np.zeros((N,N,3,Nf), dtype=np.uint8)

for i in range(Nf):
    MC.mcKernel(fM[i], fS, th[0], thS[i], fT, v, octa)
    MC.A = MC.thr.to_device(MC.al)
    MC.B = MC.thr.to_device(MC.be)
    MC.C = MC.thr.to_device(MC.spatialKernel.astype(np.complex64))
    frame = MC.getFrame() + 127.5
    frame0 = 1.0*frame
    frame[frame>255]=255
    frame[frame<0]=0
    im[:,:,0,i] = np.uint8(frame)
    im[:,:,1,i] = np.uint8(frame)
    im[:,:,2,i] = np.uint8(frame)
    
wvid = imageio.get_writer('mcTraj.avi', fps=40.0)

for i in range(Nf):
    wvid.append_data(im[:,:,:,i])
    
wvid.close()

In [102]:
# utils for artistic examples
from scipy import interpolate

def image_histogram_equalization(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html
    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, normed=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = cdf / cdf[-1] # normalize
    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)
    return image_equalized.reshape(image.shape), cdf

In [8]:
Nred = 136
# read images with some adjustements 
im1 = np.float32(imageio.imread('artistic_examples/treeSummer.png'))[16:,Nred:-Nred,:]
im2 = np.float32(imageio.imread('artistic_examples/treeWinter.png'))[16:,Nred:-Nred,:]

# init MC
fM = 1.0
fS = 0.5
octa = 1.0
th = 0.0
thS = 5.0
fT = 1.0/0.095
v = 0.0*np.array([np.cos(th),np.sin(th)])

MCinterp = motionCloud(overSamp=30, framePerSecond=20, N=256, chooseDev=0, show=1)
MCinterp.mcKernel(fM, fS, th, thS, fT, v, octa)
MCinterp.initGPU()

Correct parameters LifeTime = 0.095000 > 0.090487 



In [9]:
# First set up the figure, the axis, and the plot element we want to animate
fig02, axes02 = plt.subplots(nrows=1, ncols=1,figsize=(5, 5))
plt.close()

def init():
    return ()

def animate(i): 
    
    frameInterp = MCinterp.getFrame()
    grid = np.linspace(0,1,MCinterp.N)
    f = interpolate.RectBivariateSpline(grid,grid,MCinterp.getFrame())
    gridNew = np.linspace(0,1,im1.shape[0])
    frameInterp = f(gridNew, gridNew)
    frameInterp = (frameInterp - frameInterp.min())/(frameInterp.max()-frameInterp.min())
    
    # spike
    #frameInterp = np.maximum(1.0-np.absolute( np.mod(3.0*frameInterp,2)-1.0  ), 0.0)**3
    # sine
    #frameInterp = (np.sin(np.pi*frameInterp*5)+1.0)/2.0
    # binarize
    #frameInterp[frameInterp>0.5] = 1.0
    #frameInterp[frameInterp<=0.5] = 0.0
    
    #frameInterp = frameInterp[:,:,np.newaxis]
    
    # histogram equalization
    frameInterp, _ = image_histogram_equalization(frameInterp.ravel())
    frameInterp = frameInterp.reshape((im1.shape[0],im1.shape[0]))[:,:,np.newaxis]
    
    im0 = axes02.imshow(np.uint8(frameInterp*im1+(1.0-frameInterp)*im2))
    axes02.xaxis.set_visible(False)
    axes02.yaxis.set_visible(False)
    
    clear_output(wait=True)
    print 'It: %i'%i
    sys.stdout.flush()
    return (im0,)
            
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig02, animate, init_func=init, frames=100, interval=50, blit=True)
HTML(anim.to_html5_video()) 


It: 99


In [10]:
im = np.zeros((496,496,3,128), dtype=np.uint8)

for i in range(128):
    
    frameInterp = MCinterp.getFrame()
    grid = np.linspace(0,1,MCinterp.N)
    f = interpolate.RectBivariateSpline(grid,grid,MCinterp.getFrame())
    gridNew = np.linspace(0,1,im1.shape[0])
    frameInterp = f(gridNew, gridNew)
    
    frameInterp = (frameInterp - frameInterp.min())/(frameInterp.max()-frameInterp.min())
    # spiking
    #frameInterp = np.maximum(1.0-np.absolute( np.mod(3.0*frameInterp,2)-1.0  ), 0.0)**3
    # sine
    #frameInterp = (np.sin(np.pi*frameInterp*5.0)+1.0)/2.0
    # binarize
    #frameInterp[frameInterp>0.5] = 1.0
    #frameInterp[frameInterp<=0.5] = 0.0
    
    #frameInterp = frameInterp[:,:,np.newaxis]
    
    # histogram equalization
    frameInterp, _ = image_histogram_equalization(frameInterp.ravel())
    frameInterp = frameInterp.reshape((im1.shape[0],im1.shape[0]))[:,:,np.newaxis]
    
    
    im[:,:,:,i] = np.uint8(frameInterp*im1+(1.0-frameInterp)*im2)
    
wvid = imageio.get_writer('artistic_examples/art0.avi', fps=20.0)

for i in range(128):
    wvid.append_data(im[:,:,:,i])
    
wvid.close()